In [1]:
import config
import pandas as pd
import psycopg2
import numpy as np

- Code acte consultation : « CNP »
- Base tarif : 42,5 (Secteur 2 non OPTAM)
- MPC : majoration du médecin spécialiste : 2,7 euros
- Donc S1 et S2 OPTAM = 45,2
- keep if strpos(v17,"CNP")
- MCS: MCS : majoration de coordination pour les psychiatres : 5 euros

In [2]:
study_id = 1
sql = f"select * from depassement_study where id={study_id}"
df_study = pd.read_sql(sql, config.connection_string)
df_study

,id,profession_type,datesource_min,datesource_max,tarif_s1
0,1,Psychiatre,2301,2308,50.2


In [3]:
profession_type = df_study.iloc[0]["profession_type"]
datesource_min = df_study.iloc[0]["datesource_min"]
datesource_max = df_study.iloc[0]["datesource_max"]
tarif_s1 = df_study.iloc[0]["tarif_s1"]
profession_type, datesource_min, datesource_max, tarif_s1

('Psychiatre', 2301, 2308, 50.2)

In [4]:
sql = f"""
select p.*, t.*, tds.date_source_id, b.id as adresse_id, an.cp as cp, ar.dept_id as dept_id, b.code_insee from ps p
join tarif t on t.ps_id = p.id
join tarif_date_source tds on tds.tarif_id = t.id
join cabinet c on t.cabinet_id = c.id
join adresse_raw ar on c.adresse_raw_id = ar.id
join adresse_norm an on ar.adresse_norm_id = an.id
join ban b on an.ban_id = b.id
--join ps_cabinet_date_source pcd on pcd.ps_id = p.id and pcd.cabinet_id = c.id and pcd.date_source_id = tds.date_source_id
join profession_type pt on pt.profession = '{profession_type}' and t.profession_id = pt.profession_id
where tds.date_source_id >= {datesource_min} and  tds.date_source_id <= {datesource_max}
"""

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

df = pd.read_csv("../data/depassement/psychiatres.csv", low_memory=False)
# df = pd.read_sql(sql, config.connection_string)  #63472
df  #62172





,id,genre,key,nom,prenom,has_inpp,rule_nb,id.1,profession_id,mode_exercice_id,nature_id,convention_id,option_contrat,vitale,code,ps_id,cabinet_id,famille_acte_id,montant,borne_inf,borne_sup,montant_2,borne_inf_2,borne_sup_2,montant_imagerie,borne_inf_imagerie,borne_sup_imagerie,montant_anesthesie,borne_inf_anesthesie,borne_sup_anesthesie,montant_cec,borne_inf_cec,borne_sup_cec,tarif_id,date_source_id,id.2,nom.1,key.1,telephone,adresse_raw_id,id.3,adresse2,adresse3,adresse4,cp,commune,adresse_norm_id,dept_id,id.4,numero,rue1,rue2,cp.1,commune.1,osm_id,osm_score,ban_id,ban_score,source_id,lon,lat,score,dept_id.1,iris,id.5,adresse_id,numero.1,rep,nom_voie,code_postal,code_insee,nom_commune,nom_ancienne_commune,lon.1,lat.1,libelle_acheminement,nom_afnor,dept_id.2,is_lieu_dit
0,649908,F,810003288619,BAUDOIN,HELENE,True,1.0,15880739,65,NaN,3,1,False,True,CNP+MPC+MCS,649908,2017,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15880739,2301,2017,BAUDOIN HELENE,BAUDOIN_HELENE_06000_12_RUE_DE_PARIS,493808554.0,1610,1610,NaN,12 RUE DE PARIS,NaN,6000,NICE,1560,6,1560,12.0,RUE DE PARIS,NaN,6000,NICE,NaN,NaN,936891,1.000,2,7.268484,43.705125,1.000,6,060881704,936891,06088_4790_00012,12.0,NaN,RUE DE PARIS,6000,6088,NICE,NaN,7.268484,43.705125,NaN,NaN,6,False
1,649908,F,810003288619,BAUDOIN,HELENE,True,1.0,15880739,65,NaN,3,1,False,True,CNP+MPC+MCS,649908,2017,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15880739,2302,2017,BAUDOIN HELENE,BAUDOIN_HELENE_06000_12_RUE_DE_PARIS,493808554.0,1610,1610,NaN,12 RUE DE PARIS,NaN,6000,NICE,1560,6,1560,12.0,RUE DE PARIS,NaN,6000,NICE,NaN,NaN,936891,1.000,2,7.268484,43.705125,1.000,6,060881704,936891,06088_4790_00012,12.0,NaN,RUE DE PARIS,6000,6088,NICE,NaN,7.268484,43.705125,NaN,NaN,6,False
2,649908,F,810003288619,BAUDOIN,HELENE,True,1.0,15880739,65,NaN,3,1,False,True,CNP+MPC+MCS,649908,2017,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15880739,2303,2017,BAUDOIN HELENE,BAUDOIN_HELENE_06000_12_RUE_DE_PARIS,493808554.0,1610,1610,NaN,12 RUE DE PARIS,NaN,6000,NICE,1560,6,1560,12.0,RUE DE PARIS,NaN,6000,NICE,NaN,NaN,936891,1.000,2,7.268484,43.705125,1.000,6,060881704,936891,06088_4790_00012,12.0,NaN,RUE DE PARIS,6000,6088,NICE,NaN,7.268484,43.705125,NaN,NaN,6,False
3,649908,F,810003288619,BAUDOIN,HELENE,True,1.0,15880739,65,NaN,3,1,False,True,CNP+MPC+MCS,649908,2017,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15880739,2304,2017,BAUDOIN HELENE,BAUDOIN_HELENE_06000_12_RUE_DE_PARIS,493808554.0,1610,1610,NaN,12 RUE DE PARIS,NaN,6000,NICE,1560,6,1560,12.0,RUE DE PARIS,NaN,6000,NICE,NaN,NaN,936891,1.000,2,7.268484,43.705125,1.000,6,060881704,936891,06088_4790_00012,12.0,NaN,RUE DE PARIS,6000,6088,NICE,NaN,7.268484,43.705125,NaN,NaN,6,False
4,649908,F,810003288619,BAUDOIN,HELENE,True,1.0,15880739,65,NaN,3,1,False,True,CNP+MPC+MCS,649908,2017,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15880739,2305,2017,BAUDOIN HELENE,BAUDOIN_HELENE_06000_12_RUE_DE_PARIS,493808554.0,1610,1610,NaN,12 RUE DE PARIS,NaN,6000,NICE,1560,6,1560,12.0,RUE DE PARIS,NaN,6000,NICE,NaN,NaN,936891,1.000,2,7.268484,43.705125,1.000,6,060881704,936891,06088_4790_00012,12.0,NaN,RUE DE PARIS,6000,6088,NICE,NaN,7.268484,43.705125,NaN,NaN,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62168,729579,H,810100780583,AKHRIF,MEHDI,True,1.0,17488899,65,NaN,2,1,False,True,CNP+MPC+MCS,729579,92304,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17488899,2304,92304,AKHRIF MEHDI,AKHRIF_MEHDI_45500_RUE_DES_COTEAUX_DU_GIENNOIS,238052600.0,68457,68457,CLINIQUE DU PONT DE GIEN,RUE DES COTEAUX DU GIENNOIS,NaN,45500,GIEN,66545,45,66545,NaN,RUE DES COTEAUX DU GIENNOIS,CLINIQUE DU PONT DE

In [5]:
# Renommer les colonnes
df.rename(columns={
    'genre': 'gender',
    'prenom': 'prénom',
    'code': 'codeccamdelacte',
    'nature_id': 'naturedexercice',
    'convention_id': 'convention',
    'code_insee': 'codeinsee',
    'cp': 'matchcp',
    'montant': 'montantgénéralementconstaté',
    'borne_inf': 'borneinférieuredumontant',
    'borne_sup': 'bornesupérieuredumontant',
    'option_contrat': 'optioncontratdaccèsauxsoins',
    'dept_id': 'dep',
}, inplace=True)

# Colonnes à conserver
df = df[['ps_id', 'gender', 'nom', 'prénom', 'codeccamdelacte', 'naturedexercice', 'convention',
         'codeinsee', 'montantgénéralementconstaté', 'borneinférieuredumontant', 'bornesupérieuredumontant',
         'adresse_id', 'date_source_id', 'optioncontratdaccèsauxsoins', 'matchcp', 'dep']]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep
0,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2301,False,6000,6
1,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2302,False,6000,6
2,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2303,False,6000,6
3,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2304,False,6000,6
4,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2305,False,6000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62168,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2304,False,45500,45
62169,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2305,False,45500,45
62170,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2306,False,45500,45
62171,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2307,False,45500,45


In [6]:
# Clé combinée
pd.options.mode.copy_on_write = True
df['b'] = df['ps_id'].astype(str) + "_" + df['adresse_id'].astype(str)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b
0,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2301,False,6000,6,649908_06088_4790_00012
1,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2302,False,6000,6,649908_06088_4790_00012
2,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2303,False,6000,6,649908_06088_4790_00012
3,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2304,False,6000,6,649908_06088_4790_00012
4,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2305,False,6000,6,649908_06088_4790_00012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62168,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2304,False,45500,45,729579_45155_0412_00001
62169,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2305,False,45500,45,729579_45155_0412_00001
62170,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2306,False,45500,45,729579_45155_0412_00001
62171,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2307,False,45500,45,729579_45155_0412_00001


In [7]:
nb_ps_id = df["ps_id"].nunique()
nb_ps_id  # 5439 sql=5438


5439

In [8]:
nb_b = df["b"].nunique()
nb_b  # 5674 sql=5794

5674

In [9]:
df = df.sort_values(by='ps_id')
df['unique'] = df.groupby('ps_id')['adresse_id'].transform('nunique')
df['un'] = df['unique']
pd.set_option('display.max_rows', 200)
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,648102_01091_0331_00539,1,1
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,648102_01091_0331_00539,1,1
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,648102_01091_0331_00539,1,1
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,648102_01091_0331_00539,1,1
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,648102_01091_0331_00539,1,1
31690,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2308,False,1200,1,648102_01091_0331_00539,1,1
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,648102_01091_0331_00539,1,1
31715,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,648107_01091_0331_00539,1,1
31716,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2302,False,1200,1,648107_01091_0331_00539,1,1


In [10]:
# Comptages globaux
df['NB_total'] = df.groupby('dep')['ps_id'].transform('nunique')
df = df.sort_values(by='dep')
df["NB_total"].unique()

array([  14,   24,   10,   11,    6,  188,    8,    7,    5,   15,  346,
         40,    3,   34,   45,   28,   17,   38,   20,   26,    9,   56,
         53,  204,  289,  164,   86,   66,  118,   22,   49,  113,    2,
         58,   35,    4,   64,   41,   44,  122,   19,   32,   81,   72,
         12,   33,  145,  272,   57, 1016,   76,  102,   13,   89,   39,
         18,   29,  132,   52], dtype=int64)

In [11]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2_total'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2_total'] = df.groupby('dep')['NB2_total'].transform('max')
df["NB2_total"] = df["NB2_total"].fillna(0)
df["NB2_total"].unique()



array([  6.,   3.,  82.,   2.,   1.,   0., 134.,  16.,   5.,  13.,   7.,
         4.,  11.,  44., 116.,  39.,  23.,  28.,  38.,  18.,  57.,  25.,
        41.,  10.,  15.,  72.,  26.,   8.,  84.,  27., 179.,  35., 700.,
        31.,  65.,  33.,  87.,  14.,  43.,   9.])

In [12]:
df['c'] = 1
df['NB_Ftotal'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_Ftotal"].unique() # 5439

array([5439], dtype=int64)

In [13]:
df['NB2_Ftotal'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_Ftotal'] = df['NB2_Ftotal'].max()  # Car ne diot pas avoir de nan
df["NB2_Ftotal"].unique() # 2436

array([2436.])

In [14]:
# Pondération
df['weight'] = df['un'].map({1: 1, 2: 0.5, 3: 0.33, 4: 0.25, 5: 0.2}).fillna(0)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
60728,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2308,False,1600,1,739108_01322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0
60723,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2303,False,1600,1,739108_01322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0
57130,697644,H,SERMET,ERIC,ALQP0030,3,1,1033,69.1,0.0,0.0,01091_0331_00539,2306,True,1200,1,697644_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
57131,697644,H,SERMET,ERIC,ALQP0030,3,1,1033,69.1,0.0,0.0,01091_0331_00539,2307,True,1200,1,697644_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2306,False,20290,202,659292_2b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2702,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2308,False,20290,202,659292_2b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2701,659292,H,STALLA,PATRICK,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2307,False,20290,202,659292_2b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0
2716,659339,H,BALDOCCHI,DUMENICU,AZRP0010,3,3,2B042,39.2,0.0,0.0,2b042_0293_00005,2305,True,20290,202,659339_2b042_0293_00005,2,2,15,9.0,1,5439,2436.0,0.5


In [15]:
# Tous les praticiens ne font pas de consultation
nb_ps_id, df["ps_id"].nunique()

(5439, 5439)

In [16]:
df["codeccamdelacte"].value_counts(normalize=True)

codeccamdelacte
CNP+MPC+MCS        0.543934
CNP                0.185306
ALQP0030           0.130137
APY                0.066685
AZRP0010           0.017773
ALQP0060           0.013704
ALQP0040           0.011565
AHQP0040           0.007592
GLQP0070           0.004536
CNP+MP             0.002300
DEQP0030           0.002059
ALQP0050           0.001930
ALQP0020           0.001673
CNP+MPC+MCS+MTX    0.001415
AMQP0120           0.001287
APU                0.001287
AMQP0140           0.001158
AMQP0090           0.000772
AMQP0150           0.000643
AHQP0080           0.000386
AHQP0110           0.000386
CNP+MPC+MTX        0.000257
AHQP0120           0.000257
AHQP0030           0.000257
AHQB0250           0.000257
AMQP0130           0.000129
AHQB0020           0.000129
NZLB0010           0.000129
GLQP0120           0.000129
GLQP0080           0.000129
EQQM0030           0.000129
NZJB0010           0.000129
JLQE0020           0.000129
BDQP0030           0.000129
QZRB0010           0.000129
ANQP

In [17]:
df = df.sort_values(by='convention')
df.groupby('convention')["ps_id"].nunique()

convention
1    3084
2       2
3    2353
Name: ps_id, dtype: int64

In [18]:
df = df.sort_values(by=['b',"convention", "codeccamdelacte", "date_source_id"])
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31690,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2308,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31715,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0
31716,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2302,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0


In [19]:
# Moyenne par groupe
df['mp'] = df.groupby(['b', 'convention', 'codeccamdelacte'])['montantgénéralementconstaté'].transform('mean')
df.head(30)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31690,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2308,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31715,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,0.0
31716,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2302,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,0.0


In [20]:
# 45339
df = df[(df['codeccamdelacte'] == "CNP") | (df['codeccamdelacte'] == "CNP+MPC+MCS")]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61234,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2307,False,13009,13,799634_13209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,0.0
61235,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2308,False,13009,13,799634_13209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,0.0
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,799731_31188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,0.0
15325,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2308,False,31470,31,799731_31188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,0.0


In [21]:
tarif_s1

50.2

In [22]:
# 23421
print(len(df[(df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False)]))
df.loc[(df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False), 'mp'] = tarif_s1
df


23421


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61234,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2307,False,13009,13,799634_13209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2
61235,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2308,False,13009,13,799634_13209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,799731_31188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2
15325,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2308,False,31470,31,799731_31188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2


****on élimine les duplications ps-adresse convention acte puis on sélectionne le prix le plus élevé d'un acte de consultation (discard tarif opposable des secteurs 2)
***puis on élimine les duplications ps-adresse convention (car les actes ont le même prix maintenant pour chaque ps-adresse)

In [23]:
# Efface 39500
# Reste 5839
df = df.drop_duplicates(subset=['b', 'convention', 'codeccamdelacte'])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,648127_01266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.0,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,648312_01004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.5,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,648312_69383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.5,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30805,799569,F,MICHAUT,CAROLINE,CNP+MPC+MCS,3,3,59009,50.2,50.2,50.2,59009_6050_00001,2301,True,59650,59,799569_59009_6050_00001,1,1,122,72.0,1,5439,2436.0,1.0,50.2
61171,799615,F,RUBINSTENN,ANNE,CNP+MPC+MCS,3,1,31555,0.0,0.0,0.0,31555_6020_00005,2302,False,31400,31,799615_31555_6020_00005,1,1,204,44.0,1,5439,2436.0,1.0,50.2
61230,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2303,False,13009,13,799634_13209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,799731_31188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2


In [24]:
df = df.sort_values(by=['b',"convention"])
df.head(10)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,648127_01266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.00,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,648312_01004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.50,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,648312_69383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.50,50.2
62037,648449,H,HADDAG,NACER,CNP+MPC+MCS,3,1,2361,0.0,0.0,0.0,02361_0040_00041,2301,False,2120,2,648449_02361_0040_00041,3,3,24,3.0,1,5439,2436.0,0.33,50.2
62069,648449,H,HADDAG,NACER,CNP+MPC+MCS,4,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,648449_02691_1080_00036,3,3,24,3.0,1,5439,2436.0,0.33,50.2
31747,648503,H,KHARIJ,BRAHIM,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1850_00009,2301,False,2100,2,648503_02691_1850_00009,1,1,24,3.0,1,5439,2436.0,1.00,50.2
61991,648584,H,DELSAUX,JONATHAN,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_4370_00004,2301,False,2100,2,648584_02691_4370_00004,1,1,24,3.0,1,5439,2436.0,1.00,50.2
62053,648597,H,LENDER,PIERRE ETIENNE,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,648597_02691_1080_00036,1,1,24,3.0,1,5439,2436.0,1.00,50.2


In [25]:
df['prixmoyen'] = df.groupby(['b', 'convention'])['mp'].transform('max')
df.head(30)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,648127_01266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.00,50.2,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,648312_01004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.50,50.2,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,648312_69383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.50,50.2,50.2
62037,648449,H,HADDAG,NACER,CNP+MPC+MCS,3,1,2361,0.0,0.0,0.0,02361_0040_00041,2301,False,2120,2,648449_02361_0040_00041,3,3,24,3.0,1,5439,2436.0,0.33,50.2,50.2
62069,648449,H,HADDAG,NACER,CNP+MPC+MCS,4,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,648449_02691_1080_00036,3,3,24,3.0,1,5439,2436.0,0.33,50.2,50.2
31747,648503,H,KHARIJ,BRAHIM,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1850_00009,2301,False,2100,2,648503_02691_1850_00009,1,1,24,3.0,1,5439,2436.0,1.00,50.2,50.2
61991,648584,H,DELSAUX,JONATHAN,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_4370_00004,2301,False,2100,2,648584_02691_4370_00004,1,1,24,3.0,1,5439,2436.0,1.00,50.2,50.2
62053,648597,H,LENDER,PIERRE ETIENNE,CNP+MPC+MCS,3,1,2691,0.0,0.0,0.0,02691_1080_00036,2301,False,2100,2,648597_02691_1080_00036,1,1,24,3.0,1,5439,2436.0,1.00,50.2,50.2


In [26]:
# Reste 5522
df = df.drop_duplicates(subset=['b', 'convention'])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2
31723,648107,F,SPIRIDONOVA,LUBA,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,648107_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2
31675,648127,H,STENGER,STEPHANE,CNP+MPC+MCS,3,1,1266,0.0,0.0,0.0,01266_0065_00326,2301,False,1340,1,648127_01266_0065_00326,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2
27248,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,1004,50.2,50.2,75.0,01004_0420_00209,2301,True,1500,1,648312_01004_0420_00209,2,2,14,6.0,1,5439,2436.0,0.5,50.2,50.2
60553,648312,F,MAUSSAC,MICHELE,CNP+MPC+MCS,3,3,69383,50.2,50.2,85.0,69383_5220_00078,2301,True,69003,69,648312_69383_5220_00078,2,2,272,179.0,1,5439,2436.0,0.5,50.2,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30805,799569,F,MICHAUT,CAROLINE,CNP+MPC+MCS,3,3,59009,50.2,50.2,50.2,59009_6050_00001,2301,True,59650,59,799569_59009_6050_00001,1,1,122,72.0,1,5439,2436.0,1.0,50.2,50.2
61171,799615,F,RUBINSTENN,ANNE,CNP+MPC+MCS,3,1,31555,0.0,0.0,0.0,31555_6020_00005,2302,False,31400,31,799615_31555_6020_00005,1,1,204,44.0,1,5439,2436.0,1.0,50.2,50.2
61230,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2303,False,13009,13,799634_13209_4629_00002,1,1,346,134.0,1,5439,2436.0,1.0,50.2,50.2
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,799731_31188_2noilo_00003,1,1,204,44.0,1,5439,2436.0,1.0,50.2,50.2


In [27]:
df.groupby('convention')["ps_id"].nunique()

convention
1    3071
2       2
3    2337
Name: ps_id, dtype: int64

In [28]:
# df = df.drop_duplicates(subset=['b']) inutile
print(len(df[df["prixmoyen"] == 0])) #2
df['prixmoyen'] = df['prixmoyen'].replace(0, tarif_s1)
# ////Prix OK ////

2


In [29]:
# Calculs départementaux
df = df.sort_values(by="dep")
df['NB'] = df.groupby('dep')['ps_id'].transform('nunique')
df["NB"].unique() # Jamais == nan

array([  14,   24,   10,    6,  187,    7,    5,   15,   11,  341,   39,
          3,   34,   45,   28,   17,   38,   20,   26,    9,   56,   51,
        203,    8,  287,  164,   86,   66,  116,   22,   48,  112,    2,
         58,   35,    4,   64,   40,   44,  118,   19,   32,   81,   71,
         12,   33,  145,  270,   57, 1010,   76,  102,   13,   88,   18,
         21,   63,  131,   75], dtype=int64)

In [30]:
df['NB_F'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_F"].unique() # 5410

array([5410], dtype=int64)

In [31]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2'] = df.groupby("dep")["NB2"].transform("mean")
df["NB2"] = df["NB2"].fillna(0)
df["NB2"].unique()
df[df["NB2"] == 0]["dep"].unique()

array([10, 12, 23, 32, 46, 48, 50, 53, 55, 58, 82, 88], dtype=int64)

In [32]:
df['NB2_F'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_F'] = df['NB2_F'].mean()
df["NB2_F"].unique()  # 2418

array([2418.])

In [33]:
df['ShareS2'] = df['NB2'] / df['NB']
df['ShareS2_F'] = df['NB2_F'] / df['NB_F']
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
29318,786370,H,DJOUMESSI TEMAH,MOISE,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00244,2301,False,1000,1,786370_01344_0391_00244,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
51874,731824,H,BLONDEAU,JEAN DENIS,CNP+MPC+MCS,4,1,1034,0.0,0.0,0.0,01034_0295_00034,2301,False,1300,1,731824_01034_0295_00034,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
57133,697644,H,SERMET,ERIC,CNP+MPC+MCS,3,1,1033,50.2,50.2,50.2,01091_0331_00539,2301,True,1200,1,697644_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
60721,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2301,False,1600,1,739108_01322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
61612,795001,F,JOHN,ALTHEA MARIA,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00039,2301,False,1000,1,795001_01344_0391_00039,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
61851,788555,F,VARICLIER,SOPHIE,CNP+MPC+MCS,3,3,1033,50.2,50.2,75.0,01091_0331_00539,2301,True,1200,1,788555_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
31699,757518,H,MAURIN,ALAIN,CNP+MPC+MCS,3,3,1033,85.0,50.2,85.0,01091_0331_00539,2301,True,1200,1,757518_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,85.0,85.0,14,5410,6.0,2418.0,0.428571,0.44695
29422,788556,F,PRIOUX HABIB SABAU,CHERINE,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,788556_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695
31731,742908,F,SELLIER,CLAIRE,CNP+MPC+MCS,3,1,1033,50.2,50.2,75.0,01033_0020_00009,2301,True,1200,1,742908_01033_0020_00009,2,2,14,6.0,1,5439,2436.0,0.5,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695


In [34]:
# Dépassement
df['exessr'] = ((df['prixmoyen'] - tarif_s1) / tarif_s1) * 100
df.head(10)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
29318,786370,H,DJOUMESSI TEMAH,MOISE,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00244,2301,False,1000,1,786370_01344_0391_00244,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
51874,731824,H,BLONDEAU,JEAN DENIS,CNP+MPC+MCS,4,1,1034,0.0,0.0,0.0,01034_0295_00034,2301,False,1300,1,731824_01034_0295_00034,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
57133,697644,H,SERMET,ERIC,CNP+MPC+MCS,3,1,1033,50.2,50.2,50.2,01091_0331_00539,2301,True,1200,1,697644_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
60721,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2301,False,1600,1,739108_01322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
61612,795001,F,JOHN,ALTHEA MARIA,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00039,2301,False,1000,1,795001_01344_0391_00039,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
61851,788555,F,VARICLIER,SOPHIE,CNP+MPC+MCS,3,3,1033,50.2,50.2,75.0,01091_0331_00539,2301,True,1200,1,788555_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
31699,757518,H,MAURIN,ALAIN,CNP+MPC+MCS,3,3,1033,85.0,50.2,85.0,01091_0331_00539,2301,True,1200,1,757518_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,85.0,85.0,14,5410,6.0,2418.0,0.428571,0.44695,69.322709
29422,788556,F,PRIOUX HABIB SABAU,CHERINE,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,788556_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000
31731,742908,F,SELLIER,CLAIRE,CNP+MPC+MCS,3,1,1033,50.2,50.2,75.0,01033_0020_00009,2301,True,1200,1,742908_01033_0020_00009,2,2,14,6.0,1,5439,2436.0,0.5,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000


In [35]:
for seuil in [10, 25, 50, 75, 100]:
    df[f'c{seuil}'] = df.groupby('dep')['exessr'].transform(lambda x: (x >= seuil).sum())
    df[f'c{seuil}_F'] = (df['exessr'] >= seuil).sum()
    df[f'r{seuil}'] = (df[f'c{seuil}'] / df['NB']) * 100
    df[f'r{seuil}_F'] = df[f'c{seuil}_F'] / df['NB_F']
print("c10", df["c10"].unique())
print("c25", df["c25"].unique())
print("c50", df["c50"].unique())
print("c100", df["c100"].unique())
print("cX0_F", df["c10_F"].unique(), df["c25_F"].unique(), df["c50_F"].unique(), df["c100_F"].unique())
print("rX0_F", df["r10_F"].unique(), df["r25_F"].unique(), df["r50_F"].unique(), df["r100_F"].unique())
df.head(20)

c10 [  1   2   3  58   0  86  11   5   8   6  13   4   7  16  90  26  21  25
  28  12  50   9  37  60  14  79 166  18  35 679  36  62  24  29  82  40]
c25 [  1   2  54   3   0  82   9   5  10  13   6  14  84  23  17  25  24  12
  46  19  32   7  59  11  76 163  18  31 654  34  61   4  28   8  39]
c50 [  1   0  25  40   3   4   2  55   9   7  19  13   6  45  31  15 138  26
 586  18  50  64  28]
c100 [  0   1   7 261  10   3   4  29   5]
cX0_F [2087] [1975] [1351] [328]
rX0_F [0.3857671] [0.3650647] [0.24972274] [0.06062847]


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr,c10,c10_F,r10,r10_F,c25,c25_F,r25,r25_F,c50,c50_F,r50,r50_F,c75,c75_F,r75,r75_F,c100,c100_F,r100,r100_F
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
29318,786370,H,DJOUMESSI TEMAH,MOISE,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00244,2301,False,1000,1,786370_01344_0391_00244,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
51874,731824,H,BLONDEAU,JEAN DENIS,CNP+MPC+MCS,4,1,1034,0.0,0.0,0.0,01034_0295_00034,2301,False,1300,1,731824_01034_0295_00034,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
57133,697644,H,SERMET,ERIC,CNP+MPC+MCS,3,1,1033,50.2,50.2,50.2,01091_0331_00539,2301,True,1200,1,697644_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
60721,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2301,False,1600,1,739108_01322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
61612,795001,F,JOHN,ALTHEA MARIA,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00039,2301,False,1000,1,795001_01344_0391_00039,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
61851,788555,F,VARICLIER,SOPHIE,CNP+MPC+MCS,3,3,1033,50.2,50.2,75.0,01091_0331_00539,2301,True,1200,1,788555_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
31699,757518,H,MAURIN,ALAIN,CNP+MPC+MCS,3,3,1033,85.0,50.2,85.0,01091_0331_00539,2301,True,1200,1,757518_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,85.0,85.0,14,5410,6.0,2418.0,0.428571,0.44695,69.322709,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
29422,788556,F,PRIOUX HABIB SABAU,CHERINE,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,788556_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628
31731,742908,F,SELLIER,CLAIRE,CNP+MPC+MCS,3,1,1033,50.2,50.2,75.0,01033_0020_00009,2301,True,1200,1,742908_01033_0020_00009,2,2,14,6.0,1,5439,2436.0,0.5,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628


****dépassement moyen par département (uniquement les acteurs  faisant du dépassement)

In [36]:
# Moyennes nationales
df['PF'] = df['prixmoyen'].mean() # 63.86
df["PF"].unique()

array([63.86873642])

In [37]:
df['PFS2'] = df[mask_s2]['prixmoyen'].mean()
df['PFS2'].unique()  # 80.45

array([80.45200902])

In [38]:
# Moyennes départementales
df['PrixMoyen'] = df.groupby('dep')['prixmoyen'].transform('mean')
df['PrixMoyen'].unique()

array([52.68571429, 51.39615385, 52.18      , 52.16      , 58.81666667,
       58.46422872, 56.58571429, 57.72727273, 50.2       , 51.52      ,
       56.76875   , 56.145     , 60.13333333, 55.84285714, 56.89595588,
       52.96      , 51.86      , 53.36444444, 54.30344828, 50.2       ,
       53.42941176, 55.48684211, 57.54      , 62.40185185, 63.48888889,
       51.55178571, 52.68431373, 51.94807692, 58.28886986, 53.86969697,
       55.36627907, 56.64285714, 63.71176471, 55.34576271, 50.73333333,
       51.2       , 51.15909091, 56.275     , 53.90714286, 59.65446429,
       54.        , 53.07586207, 65.14864865, 53.275     , 64.6       ,
       54.9804878 , 57.53111111, 66.48583333, 58.50526316, 53.13333333,
       54.85625   , 52.26341463, 53.52916667, 53.27352941, 63.50204082,
       65.74666667, 71.41829044, 56.72      , 57.4       , 51.78181818,
       62.09705882, 67.81525424, 83.62432039, 62.30759494, 64.45166667,
       74.23980583, 56.28571429, 52.59411765, 51.61428571, 59.03

In [39]:
df['PrixMoyenS2'] = df[mask_s2].groupby('dep')['prixmoyen'].transform('mean')
df.head(20)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr,c10,c10_F,r10,r10_F,c25,c25_F,r25,r25_F,c50,c50_F,r50,r50_F,c75,c75_F,r75,r75_F,c100,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,NaN
29318,786370,H,DJOUMESSI TEMAH,MOISE,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00244,2301,False,1000,1,786370_01344_0391_00244,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,NaN
51874,731824,H,BLONDEAU,JEAN DENIS,CNP+MPC+MCS,4,1,1034,0.0,0.0,0.0,01034_0295_00034,2301,False,1300,1,731824_01034_0295_00034,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,NaN
57133,697644,H,SERMET,ERIC,CNP+MPC+MCS,3,1,1033,50.2,50.2,50.2,01091_0331_00539,2301,True,1200,1,697644_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.000
60721,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2301,False,1600,1,739108_01322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,NaN
61612,795001,F,JOHN,ALTHEA MARIA,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00039,2301,False,1000,1,795001_01344_0391_00039,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,NaN
61851,788555,F,VARICLIER,SOPHIE,CNP+MPC+MCS,3,3,1033,50.2,50.2,75.0,01091_0331_00539,2301,True,1200,1,788555_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.000
31699,757518,H,MAURIN,ALAIN,CNP+MPC+MCS,3,3,1033,85.0,50.2,85.0,01091_0331_00539,2301,True,1200,1,757518_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,85.0,85.0,14,5410,6.0,2418.0,0.428571,0.44695,69.322709,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.000
29422,788556,F,PRIOUX HABIB SABAU,CHERINE,CNP+MPC+MCS,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,788556_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,NaN
31731,742908,F,SELLIER,CLAIRE,CNP+MPC+MCS,3,1,1033,50.2,50.2,75.0,01033_0020_00009,2301,True,1200,1,742908_01033_0020_00009,2,2,14,6.0,1,5439,2436.0,0.5,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,

In [40]:
# df['Prix'] = df.groupby('dep')['PrixMoyen'].transform('mean')
df['depmoyen'] = ((df['PrixMoyen'] - tarif_s1) / tarif_s1) * 100  # * 100.roound(5)
df["depmoyen"].unique()

array([4.95162208e+00, 2.38277659e+00, 3.94422311e+00, 3.90438247e+00,
       1.71646746e+01, 1.64626070e+01, 1.27205464e+01, 1.49945672e+01,
       0.00000000e+00, 2.62948207e+00, 1.30851594e+01, 1.18426295e+01,
       1.97875166e+01, 1.12407513e+01, 1.33385575e+01, 5.49800797e+00,
       3.30677291e+00, 6.30367419e+00, 8.17419975e+00, 1.41542378e-14,
       6.43309116e+00, 1.05315580e+01, 1.46215139e+01, 2.43064778e+01,
       2.64718902e+01, 2.69280023e+00, 4.94883212e+00, 3.48222495e+00,
       1.61132866e+01, 7.31015333e+00, 1.02913926e+01, 1.28343768e+01,
       2.69158659e+01, 1.02505233e+01, 1.06241700e+00, 1.99203187e+00,
       1.91053966e+00, 1.21015936e+01, 7.38474673e+00, 1.88335942e+01,
       7.56972112e+00, 5.72880890e+00, 2.97781846e+01, 6.12549801e+00,
       2.86852590e+01, 9.52288407e+00, 1.46038070e+01, 3.24418991e+01,
       1.65443489e+01, 5.84329349e+00, 9.27539841e+00, 4.11038772e+00,
       6.63180611e+00, 6.12256855e+00, 2.64980893e+01, 3.09694555e+01,
      

In [41]:
#27.22
df["depmoyen_F"] =((df['PF'] - tarif_s1) / tarif_s1) * 100
df["depmoyen_F"] = df["depmoyen_F"].round(3)
df["depmoyen_F"].unique()

array([27.229])

In [42]:
df["PrixMoyenS2"] = df.groupby('dep')['PrixMoyenS2'].transform('mean')
df['depmoyenS2'] = ((df['PrixMoyenS2'] - tarif_s1) / tarif_s1) * 100
df['depmoyenS2'] = df['depmoyenS2'].round(3)
df['depmoyenS2'].unique()

array([11.554, 15.488, 13.147, 13.015, 34.329, 37.744, 29.681, 54.98 ,
        0.   ,    nan, 19.721, 33.867, 29.607, 59.363, 31.474, 34.885,
       27.49 , 16.534, 21.82 , 33.865, 27.341, 36.382, 48.738, 46.877,
       79.416, 30.159, 25.239, 16.096, 39.874, 30.928, 38.481, 29.947,
       61.009, 32.691,  9.562, 29.88 , 14.011, 32.271, 51.693, 37.006,
       28.116, 25.559, 44.072, 24.502, 44.777, 35.494, 43.811, 55.615,
       44.906, 11.687, 26.983, 33.705, 17.685, 26.021, 45.826, 51.616,
       64.228, 32.47 , 57.371, 23.108, 44.765, 55.955, 95.916, 46.473,
       50.1  , 74.734, 28.287, 23.164,  6.574, 48.375, 40.474, 45.1  ,
       26.345, 34.13 ,  8.423, 56.234, 85.421, 31.559, 67.636, 61.51 ,
       14.741, 18.127])

In [43]:
df['depmoyen_FS2'] = ((df['PFS2'] - tarif_s1) / tarif_s1) * 100
df['depmoyen_FS2'].unique() # 60.26


array([60.26296617])

In [44]:
df = df.sort_values(by=['dep', 'depmoyen'], ascending=[True, False])
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr,c10,c10_F,r10,r10_F,c25,c25_F,r25,r25_F,c50,c50_F,r50,r50_F,c75,c75_F,r75,r75_F,c100,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,648102_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.0,4.951622,27.229,11.554,60.262966
29318,786370,H,DJOUMESSI TEMAH,MOISE,CNP+MPC+MCS,3,1,1344,0.0,0.0,0.0,01344_0391_00244,2301,False,1000,1,786370_01344_0391_00244,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.0,4.951622,27.229,11.554,60.262966
51874,731824,H,BLONDEAU,JEAN DENIS,CNP+MPC+MCS,4,1,1034,0.0,0.0,0.0,01034_0295_00034,2301,False,1300,1,731824_01034_0295_00034,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.0,4.951622,27.229,11.554,60.262966
57133,697644,H,SERMET,ERIC,CNP+MPC+MCS,3,1,1033,50.2,50.2,50.2,01091_0331_00539,2301,True,1200,1,697644_01091_0331_00539,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.0,4.951622,27.229,11.554,60.262966
60721,739108,F,CHIRON,EMMANUELLE,CNP+MPC+MCS,3,1,1322,0.0,0.0,0.0,01322_0240_00127,2301,False,1600,1,739108_01322_0240_00127,1,1,14,6.0,1,5439,2436.0,1.0,50.2,50.2,14,5410,6.0,2418.0,0.428571,0.44695,0.000000,1,2087,7.142857,0.385767,1,1975,7.142857,0.365065,1,1351,7.142857,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,52.685714,56.0,4.951622,27.229,11.554,60.262966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34408,659341,H,LODOVIGHI,MARC ANTOINE,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0654_00008,2301,False,20290,202,659341_2b042_0654_00008,1,1,15,9.0,1,5439,2436.0,1.0,50.2,50.2,15,5410,9.0,2418.0,0.600000,0.44695,0.000000,4,2087,26.666667,0.385767,4,1975,26.666667,0.365065,0,1351,0.000000,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,55.887500,59.3,11.329681,27.229,18.127,60.262966
2703,659367,H,SANCHEZ,SERGE,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2301,False,20290,202,659367_2b042_0293_00005,1,1,15,9.0,1,5439,2436.0,1.0,50.2,50.2,15,5410,9.0,2418.0,0.600000,0.44695,0.000000,4,2087,26.666667,0.385767,4,1975,26.666667,0.365065,0,1351,0.000000,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,55.887500,59.3,11.329681,27.229,18.127,60.262966
43934,768998,H,PAYEN LOGLI,ARNAUD,CNP,2,3,2B033,74.8,74.8,77.5,2b033_0952_00001,2301,False,20200,202,768998_2b033_0952_00001,1,1,15,9.0,1,5439,2436.0,1.0,74.8,74.8,15,5410,9.0,2418.0,0.600000,0.44695,49.003984,4,2087,26.666667,0.385767,4,1975,26.666667,0.365065,0,1351,0.000000,0.249723,0,802,0.0,0.148244,0,328,0.0,0.060628,63.868736,80.452009,55.887500,59.3,11.329681,27.229,18.127,60.262966
2671,659254,H,DUCOMMUN,PIERRE,CNP+MPC+MCS,3,1,2B042,0.0,0.0,0.0,2b042_0293_00005,2301,Fal

In [45]:
df = df.drop_duplicates(subset=['dep'])
df['depmoyen'] = df['depmoyen'].fillna(0)
pd.set_option('display.max_rows', 20)
df['depmoyen'].unique()

array([4.95162208e+00, 2.38277659e+00, 3.94422311e+00, 3.90438247e+00,
       1.71646746e+01, 1.64626070e+01, 1.27205464e+01, 1.49945672e+01,
       0.00000000e+00, 2.62948207e+00, 1.30851594e+01, 1.18426295e+01,
       1.97875166e+01, 1.12407513e+01, 1.33385575e+01, 5.49800797e+00,
       3.30677291e+00, 6.30367419e+00, 8.17419975e+00, 1.41542378e-14,
       6.43309116e+00, 1.05315580e+01, 1.46215139e+01, 2.43064778e+01,
       2.64718902e+01, 2.69280023e+00, 4.94883212e+00, 3.48222495e+00,
       1.61132866e+01, 7.31015333e+00, 1.02913926e+01, 1.28343768e+01,
       2.69158659e+01, 1.02505233e+01, 1.06241700e+00, 1.99203187e+00,
       1.91053966e+00, 1.21015936e+01, 7.38474673e+00, 1.88335942e+01,
       7.56972112e+00, 5.72880890e+00, 2.97781846e+01, 6.12549801e+00,
       2.86852590e+01, 9.52288407e+00, 1.46038070e+01, 3.24418991e+01,
       1.65443489e+01, 5.84329349e+00, 9.27539841e+00, 4.11038772e+00,
       6.63180611e+00, 6.12256855e+00, 2.64980893e+01, 3.09694555e+01,
      

In [46]:
# gender nom prénom naturedexercice convention optioncontratdaccèsauxsoins codeccamdelacte ps_id montantgénéralementconstaté borneinférieuredumontant bornesupérieuredumontant date_source_id adresse_id matchcp codeinsee b unique un weight mp prixmoyen exessr nb10 c10 nb25 c25 nb50 c50 nb75 c75 nb100 c100 exess
df = df.drop(["gender","nom","prénom","naturedexercice","convention","optioncontratdaccèsauxsoins","codeccamdelacte","ps_id","montantgénéralementconstaté","borneinférieuredumontant","bornesupérieuredumontant","date_source_id","adresse_id","matchcp","codeinsee","b","unique","un","weight","mp","prixmoyen","exessr","c10","c25","c50","c75","c100"], axis=1)
df

,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2
31683,1,14,6.0,1,5439,2436.0,14,5410,6.0,2418.0,0.428571,0.44695,2087,7.142857,0.385767,1975,7.142857,0.365065,1351,7.142857,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.685714,56.000000,4.951622,27.229,11.554,60.262966
61999,2,24,3.0,1,5439,2436.0,24,5410,3.0,2418.0,0.125000,0.44695,2087,8.333333,0.385767,1975,4.166667,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,51.396154,57.975000,2.382777,27.229,15.488,60.262966
42707,3,10,3.0,1,5439,2436.0,10,5410,3.0,2418.0,0.300000,0.44695,2087,10.000000,0.385767,1975,10.000000,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.180000,56.800000,3.944223,27.229,13.147,60.262966
43639,4,11,3.0,1,5439,2436.0,10,5410,3.0,2418.0,0.300000,0.44695,2087,10.000000,0.385767,1975,10.000000,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.160000,56.733333,3.904382,27.229,13.015,60.262966
60345,5,6,3.0,1,5439,2436.0,6,5410,3.0,2418.0,0.500000,0.44695,2087,50.000000,0.385767,1975,33.333333,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,58.816667,67.433333,17.164675,27.229,34.329,60.262966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30741,93,58,14.0,1,5439,2436.0,57,5410,14.0,2418.0,0.245614,0.44695,2087,15.789474,0.385767,1975,14.035088,0.365065,1351,10.526316,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,54.091228,66.042857,7.751450,27.229,31.559,60.262966
56549,94,76,43.0,1,5439,2436.0,75,5410,43.0,2418.0,0.573333,0.44695,2087,53.333333,0.385767,1975,52.000000,0.365065,1351,37.333333,0.249723,802,29.333333,0.148244,328,6.666667,0.060628,63.868736,80.452009,69.666667,84.153488,38.778220,27.229,67.636,60.262966
61123,95,52,28.0,1,5439,2436.0,51,5410,27.0,2418.0,0.529412,0.44695,2087,47.058824,0.385767,1975,47.058824,0.365065,1351,37.254902,0.249723,802,13.725490,0.148244,328,1.960784,0.060628,63.868736,80.452009,66.547059,81.077778,32.563862,27.229,61.510,60.262966
60097,201,14,2.0,1,5439,2436.0,14,5410,2.0,2418.0,0.142857,0.44695,2087,7.142857,0.385767,1975,7.142857,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,51.257143,57.600000,2.105862,27.229,14.741,60.262966


In [47]:
# Séparer les lignes à dupliquer
df_to_duplicate = df[df['dep'] == 75].copy()
df_to_duplicate['dup'] = 1  # marquer les duplications
# Marquer les lignes originales
df['dup'] = 0
# Fusionner les deux
df = pd.concat([df, df_to_duplicate], ignore_index=True)
df



,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2,dup
0,1,14,6.0,1,5439,2436.0,14,5410,6.0,2418.0,0.428571,0.44695,2087,7.142857,0.385767,1975,7.142857,0.365065,1351,7.142857,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.685714,56.000000,4.951622,27.229,11.554,60.262966,0
1,2,24,3.0,1,5439,2436.0,24,5410,3.0,2418.0,0.125000,0.44695,2087,8.333333,0.385767,1975,4.166667,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,51.396154,57.975000,2.382777,27.229,15.488,60.262966,0
2,3,10,3.0,1,5439,2436.0,10,5410,3.0,2418.0,0.300000,0.44695,2087,10.000000,0.385767,1975,10.000000,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.180000,56.800000,3.944223,27.229,13.147,60.262966,0
3,4,11,3.0,1,5439,2436.0,10,5410,3.0,2418.0,0.300000,0.44695,2087,10.000000,0.385767,1975,10.000000,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.160000,56.733333,3.904382,27.229,13.015,60.262966,0
4,5,6,3.0,1,5439,2436.0,6,5410,3.0,2418.0,0.500000,0.44695,2087,50.000000,0.385767,1975,33.333333,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,58.816667,67.433333,17.164675,27.229,34.329,60.262966,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,76,43.0,1,5439,2436.0,75,5410,43.0,2418.0,0.573333,0.44695,2087,53.333333,0.385767,1975,52.000000,0.365065,1351,37.333333,0.249723,802,29.333333,0.148244,328,6.666667,0.060628,63.868736,80.452009,69.666667,84.153488,38.778220,27.229,67.636,60.262966,0
93,95,52,28.0,1,5439,2436.0,51,5410,27.0,2418.0,0.529412,0.44695,2087,47.058824,0.385767,1975,47.058824,0.365065,1351,37.254902,0.249723,802,13.725490,0.148244,328,1.960784,0.060628,63.868736,80.452009,66.547059,81.077778,32.563862,27.229,61.510,60.262966,0
94,201,14,2.0,1,5439,2436.0,14,5410,2.0,2418.0,0.142857,0.44695,2087,7.142857,0.385767,1975,7.142857,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,51.257143,57.600000,2.105862,27.229,14.741,60.262966,0
95,202,15,9.0,1,5439,2436.0,15,5410,9.0,2418.0,0.600000,0.44695,2087,26.666667,0.385767,1975,26.666667,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,55.887500,59.300000,11.329681,27.229,18.127,60.262966,0


In [48]:
df.loc[df["dup"] == 1, "dep"] = 0
df.loc[df["dup"] == 1, "NB_total"] = df.loc[df["dup"] == 1]["NB_Ftotal"]
df.loc[df["dup"] == 1, "NB2_total"] = df.loc[df["dup"] == 1]["NB2_Ftotal"]
df.loc[df["dup"] == 1, "NB"] = df.loc[df["dup"] == 1]["NB_F"]
df.loc[df["dup"] == 1, "NB2"] = df.loc[df["dup"] == 1]["NB2_F"]
df.loc[df["dup"] == 1, "ShareS2"] = df.loc[df["dup"] == 1]["ShareS2_F"]
df.loc[df["dup"] == 1, "r10"] = df.loc[df["dup"] == 1]["r10_F"]
df.loc[df["dup"] == 1, "r25"] = df.loc[df["dup"] == 1]["r25_F"]
df.loc[df["dup"] == 1, "r50"] = df.loc[df["dup"] == 1]["r50_F"]
df.loc[df["dup"] == 1, "r100"] = df.loc[df["dup"] == 1]["r100_F"]
df.loc[df["dup"] == 1, "PrixMoyen"] = df.loc[df["dup"] == 1]["PF"]
df.loc[df["dup"] == 1, "PrixMoyenS2"] = df.loc[df["dup"] == 1]["PFS2"]
df.loc[df["dup"] == 1, "depmoyen"] = df.loc[df["dup"] == 1]["depmoyen_F"]
df.loc[df["dup"] == 1, "depmoyenS2"] = df.loc[df["dup"] == 1]["depmoyen_FS2"]
df

,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2,dup
0,1,14,6.0,1,5439,2436.0,14,5410,6.0,2418.0,0.428571,0.44695,2087,7.142857,0.385767,1975,7.142857,0.365065,1351,7.142857,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.685714,56.000000,4.951622,27.229,11.554000,60.262966,0
1,2,24,3.0,1,5439,2436.0,24,5410,3.0,2418.0,0.125000,0.44695,2087,8.333333,0.385767,1975,4.166667,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,51.396154,57.975000,2.382777,27.229,15.488000,60.262966,0
2,3,10,3.0,1,5439,2436.0,10,5410,3.0,2418.0,0.300000,0.44695,2087,10.000000,0.385767,1975,10.000000,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.180000,56.800000,3.944223,27.229,13.147000,60.262966,0
3,4,11,3.0,1,5439,2436.0,10,5410,3.0,2418.0,0.300000,0.44695,2087,10.000000,0.385767,1975,10.000000,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,52.160000,56.733333,3.904382,27.229,13.015000,60.262966,0
4,5,6,3.0,1,5439,2436.0,6,5410,3.0,2418.0,0.500000,0.44695,2087,50.000000,0.385767,1975,33.333333,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,58.816667,67.433333,17.164675,27.229,34.329000,60.262966,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,76,43.0,1,5439,2436.0,75,5410,43.0,2418.0,0.573333,0.44695,2087,53.333333,0.385767,1975,52.000000,0.365065,1351,37.333333,0.249723,802,29.333333,0.148244,328,6.666667,0.060628,63.868736,80.452009,69.666667,84.153488,38.778220,27.229,67.636000,60.262966,0
93,95,52,28.0,1,5439,2436.0,51,5410,27.0,2418.0,0.529412,0.44695,2087,47.058824,0.385767,1975,47.058824,0.365065,1351,37.254902,0.249723,802,13.725490,0.148244,328,1.960784,0.060628,63.868736,80.452009,66.547059,81.077778,32.563862,27.229,61.510000,60.262966,0
94,201,14,2.0,1,5439,2436.0,14,5410,2.0,2418.0,0.142857,0.44695,2087,7.142857,0.385767,1975,7.142857,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,51.257143,57.600000,2.105862,27.229,14.741000,60.262966,0
95,202,15,9.0,1,5439,2436.0,15,5410,9.0,2418.0,0.600000,0.44695,2087,26.666667,0.385767,1975,26.666667,0.365065,1351,0.000000,0.249723,802,0.000000,0.148244,328,0.000000,0.060628,63.868736,80.452009,55.887500,59.300000,11.329681,27.229,18.127000,60.262966,0


In [49]:
# NB_Ftotal  NB2_Ftotal NB_F NB2_F ShareS2_F r10_F r25_F r50_F r100_F PF PFS2 depmoyen_F depmoyen_FS2 c10_F c25_F c50_F c100_F dup c
df = df.drop(["NB_Ftotal","NB2_Ftotal","NB_F","NB2_F","ShareS2_F","r10_F","r25_F","r50_F","r100_F","PF","PFS2","depmoyen_F","depmoyen_FS2","c10_F","c25_F","c50_F","c75_F","r75_F", "r75","c100_F","dup","c"], axis=1)
df

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
0,1,14,6.0,14,6.0,0.428571,7.142857,7.142857,7.142857,0.000000,52.685714,56.000000,4.951622,11.554000
1,2,24,3.0,24,3.0,0.125000,8.333333,4.166667,0.000000,0.000000,51.396154,57.975000,2.382777,15.488000
2,3,10,3.0,10,3.0,0.300000,10.000000,10.000000,0.000000,0.000000,52.180000,56.800000,3.944223,13.147000
3,4,11,3.0,10,3.0,0.300000,10.000000,10.000000,0.000000,0.000000,52.160000,56.733333,3.904382,13.015000
4,5,6,3.0,6,3.0,0.500000,50.000000,33.333333,0.000000,0.000000,58.816667,67.433333,17.164675,34.329000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,76,43.0,75,43.0,0.573333,53.333333,52.000000,37.333333,6.666667,69.666667,84.153488,38.778220,67.636000
93,95,52,28.0,51,27.0,0.529412,47.058824,47.058824,37.254902,1.960784,66.547059,81.077778,32.563862,61.510000
94,201,14,2.0,14,2.0,0.142857,7.142857,7.142857,0.000000,0.000000,51.257143,57.600000,2.105862,14.741000
95,202,15,9.0,15,9.0,0.600000,26.666667,26.666667,0.000000,0.000000,55.887500,59.300000,11.329681,18.127000


In [50]:
# df[df["dep"] == 38]
df = df.reset_index(drop=True)
df.head(10)


,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
0,1,14,6.0,14,6.0,0.428571,7.142857,7.142857,7.142857,0.000000,52.685714,56.000000,4.951622,11.554
1,2,24,3.0,24,3.0,0.125000,8.333333,4.166667,0.000000,0.000000,51.396154,57.975000,2.382777,15.488
2,3,10,3.0,10,3.0,0.300000,10.000000,10.000000,0.000000,0.000000,52.180000,56.800000,3.944223,13.147
3,4,11,3.0,10,3.0,0.300000,10.000000,10.000000,0.000000,0.000000,52.160000,56.733333,3.904382,13.015
4,5,6,3.0,6,3.0,0.500000,50.000000,33.333333,0.000000,0.000000,58.816667,67.433333,17.164675,34.329
5,6,188,82.0,187,82.0,0.438503,31.016043,28.877005,13.368984,0.534759,58.464229,69.147256,16.462607,37.744
6,7,8,3.0,7,3.0,0.428571,28.571429,28.571429,0.000000,0.000000,56.585714,65.100000,12.720546,29.681
7,8,10,2.0,10,2.0,0.200000,30.000000,30.000000,10.000000,0.000000,57.727273,77.800000,14.994567,54.980
8,9,7,1.0,7,1.0,0.142857,0.000000,0.000000,0.000000,0.000000,50.200000,50.200000,0.000000,0.000
9,10,5,0.0,5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,50.200000,NaN,0.000000,NaN


In [51]:
df2 = pd.read_csv("../data/depassement/Rendus_PartIII/dépassement_psychiatres.csv")
df2.head(10)

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,depmoyen,PrixMoyenS2,depmoyenS2
0,1,14,6,14,6,0.428571,7.142857,7.142857,7.142857,0.000000,52.685715,4.952,56.000000,11.554
1,2,24,3,24,3,0.125000,8.333333,4.166666,0.000000,0.000000,51.396156,2.383,57.974998,15.488
2,3,10,3,10,3,0.300000,10.000000,10.000000,0.000000,0.000000,52.180000,3.944,56.799999,13.147
3,4,11,3,10,3,0.300000,10.000000,10.000000,0.000000,0.000000,52.160000,3.904,56.733334,13.015
4,5,6,3,6,3,0.500000,50.000000,33.333332,0.000000,0.000000,58.816666,17.165,67.433334,34.329
5,6,188,82,187,82,0.438503,31.016043,28.877005,13.368984,0.534759,58.464230,16.463,69.147255,37.744
6,7,8,3,7,3,0.428571,28.571428,28.571428,0.000000,0.000000,56.585716,12.721,65.099998,29.681
7,8,10,2,10,2,0.200000,30.000000,30.000000,10.000000,0.000000,57.727272,14.995,77.799995,54.980
8,9,7,1,7,1,0.142857,0.000000,0.000000,0.000000,0.000000,50.200001,0.000,50.200001,0.000
9,10,5,0,5,0,0.000000,0.000000,0.000000,0.000000,0.000000,50.200001,0.000,NaN,NaN


In [52]:
df1 = df.fillna(9999).round(2)
df2 = df2.fillna(9999).round(2)
diff = df1 - df2
pd.set_option('display.max_rows', 100)
diff.head(96)


,NB,NB2,NB2_total,NB_total,PrixMoyen,PrixMoyenS2,ShareS2,dep,depmoyen,depmoyenS2,r10,r100,r25,r50
0,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0,0.0,0.01,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
5,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
6,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
7,0,0.0,0.0,0,0.0,0.00,0.0,0,-0.01,0.0,0.0,0.0,0.0,0.0
8,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0
9,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0


In [53]:
diff = df1["NB"].equals(df2["NB"])
diff = df1[df1["NB2"] != df2["NB2"]]
diff = df1[df1["NB2_total"] != df2["NB2_total"]]
diff = df1[df1["NB_total"] != df2["NB_total"]]
diff = df1[df1["PrixMoyen"] != df2["PrixMoyen"]]
diff = df1[df1["PrixMoyenS2"] != df2["PrixMoyenS2"]] # Aucune différence avec STATA, petite différence avec le fichier, + grosse différence avec SQL
diff

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
1,2,24,3.0,24,3.0,0.12,8.33,4.17,0.0,0.0,51.4,57.98,2.38,15.49


In [54]:
df.to_csv(f"../data/depassement/out/depassement_{profession_type}_{datesource_min}_{datesource_max}_{tarif_s1}_{study_id}.csv", index=False)

In [55]:
conn = psycopg2.connect(config.connection_string)
sql = f"delete from depassement where depassement_study_id={study_id}" # Ne marche pas la 1ère fois
with conn.cursor() as cur:
    cur.execute(sql)
conn.commit()
conn.close()
df["depassement_study_id"] = study_id
df.to_sql("depassement", config.connection_string, if_exists="append", index_label="id")


97